In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
pip uninstall keras

Uninstalling Keras-2.4.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/Keras-2.4.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/docs/*
    /usr/local/lib/python3.7/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.7/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.4.3


In [3]:
pip uninstall tensorflow

Uninstalling tensorflow-2.4.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.4.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.4.1


In [4]:
pip install keras==2.0.6


     |████████████████████████████████| 235kB 6.8MB/s 
  Created wheel for keras: filename=Keras-2.0.6-cp37-none-any.whl size=266418 sha256=c9fc6af851486ca7a787e0b1a1f05e5f70796b2d8a358230d7398a07e7753bb0
  Stored in directory: /root/.cache/pip/wheels/d1/70/83/9be5aef9c4c863ea21adacd0be83139b20d3d819401a2b07d3
Successfully built keras
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.6 which is incompatible.


In [5]:
pip install tensorflow==1.13.0rc1

     |████████████████████████████████| 92.7MB 66kB/s 
     |████████████████████████████████| 368kB 43.9MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 3.1MB 52.8MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1


In [6]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Cropping2D, ZeroPadding2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
from keras import backend as K
import os
from skimage.transform import resize
from skimage.io import imsave
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [7]:
working_path = 'gdrive/My Drive/luna/output/'
main_path = 'gdrive/My Drive/luna/'
unet_weight = 'gdrive/My Drive/luna/unet.hdf5'

In [8]:
K.set_image_data_format('channels_first')  # Theano dimension ordering in this code

BATCH_SIZE=8
EPOCHS=5
img_rows = 512
img_cols = 512

smooth = 1.
 

In [9]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [10]:
def dice_coef_np(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

In [11]:
def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

In [12]:
def get_model():

      inputs = Input((1, img_rows, img_cols))

      conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
      conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
      pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

      conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
      conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
      pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

      conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
      conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
      pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

      conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
      conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
      pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

      conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
      conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

      up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=1)
      conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
      conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

      up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=1)
      conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
      conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

      up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=1)
      conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
      conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

      up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=1)
      conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
      conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
      conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

      model = Model(inputs=inputs, outputs=conv10)
      print (model.summary())
      print(len(model.layers))
      #     model.compile(optimizer=Adam(lr=1.0e-5), loss=dice_coef_loss, metrics=[dice_coef])
      model.compile(optimizer=Adam(lr=1e-5), loss='binary_crossentropy', metrics = ['accuracy'])
      return model


In [13]:
def get_unet1():
    
      K.set_image_data_format('channels_first')  # Theano dimension ordering in this code
      inputs = Input((1, img_rows, img_cols))

      conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
      conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
      pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

      conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
      conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
      pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

      conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
      conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
      pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

      conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
      conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
      pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

      conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
      conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

      up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=1)
      conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
      conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

      up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=1)
      conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
      conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

      up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=1)
      conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
      conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

      up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=1)
      conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
      conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

      conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

      model = Model(inputs=[inputs], outputs=[conv10])
      print(len(model.layers))
      print (model.summary())
      #model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])
      model.compile(optimizer=Adam(lr=1e-5), loss='binary_crossentropy', metrics = ['accuracy'])

      return model

In [14]:
def get_unet():

      K.set_image_data_format('channels_first')  # Theano dimension ordering in this code
      inputs = Input((1, img_rows, img_cols))

      conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
      conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
      pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

      conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
      conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
      pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

      conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
      conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
      pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

      conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
      conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
      pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

      conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
      conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

      up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=1)
      conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
      conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

      up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=1)
      conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
      conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

      up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=1)
      conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
      conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

      up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=1)
      conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
      conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

      conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

      model = Model(inputs=[inputs], outputs=[conv10])
      print (model.summary())
      #model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])
      model.compile(optimizer=Adam(lr=1e-5), loss='binary_crossentropy', metrics = ['accuracy'])
      print(len(model.layers))
      return model


In [15]:
def train_and_predict(use_existing):
    print('-' * 30)
    print('Loading and preprocessing train data...')
    print ('BATCH_SIZE : {}'.format(BATCH_SIZE))
    print ('EPOCHS : {}'.format(EPOCHS))
    imgs_train = np.load(main_path + "trainImages.npy").astype(np.float32)
    imgs_mask_train = np.load(main_path + "trainMasks.npy").astype(np.float32)
    
#     imgs_val = np.load(main_path + "valImages.npy").astype(np.float32)
#     imgs_mask_val = np.load(main_path + "valMasks.npy").astype(np.float32)
    
    imgs_test = np.load(main_path + "testImages.npy").astype(np.float32)
    imgs_mask_test_true = np.load(main_path + "testMasks.npy").astype(np.float32)

    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization

    imgs_train -= mean  # images should already be standardized, but just in case
    imgs_train /= std

    print('-' * 30)
    print('Creating and compiling model...')

    model = get_model()
    # Saving weights to unet.hdf5 at checkpoints

    best_weight_path = 'gdrive/My Drive/luna/unet.hdf5'
    model_checkpoint = ModelCheckpoint(best_weight_path, monitor='val_loss', save_best_only=True)
    tb = TensorBoard(log_dir="../logs_281118", batch_size=BATCH_SIZE)

    best_weight_path = 'gdrive/My Drive/luna/unet.hdf5'.format(BATCH_SIZE)
    model_checkpoint = ModelCheckpoint(best_weight_path, monitor='val_loss', save_best_only=True)
    tb = TensorBoard(log_dir='gdrive/My Drive/luna/', batch_size=BATCH_SIZE)

    # Set argument for call to train_and_predict to true at end of script
    if use_existing:
        print('loading weights...')
        model.load_weights(unet_weight)

    print('-' * 30)
    print('Fitting model...')

    model.fit(imgs_train, imgs_mask_train, 
              validation_split=0.15,
              batch_size=BATCH_SIZE, 
              epochs=EPOCHS, 
              verbose=1, shuffle=True,
              callbacks=[model_checkpoint,tb])


    # loading best weights from training session
    print('-' * 30)
    print('Loading saved weights...')


    model.load_weights(unet_weight)

    model.load_weights(best_weight_path)


    print('-' * 30)
    print('Predicting masks on test data...')

    num_test = len(imgs_test)
    imgs_mask_test = np.ndarray([num_test, 1, 512, 512], dtype=np.float32)
    for i in range(num_test):
        imgs_mask_test[i] = model.predict([imgs_test[i:i + 1]], verbose=0)[0]

    np.save('gdrive/My Drive/luna/masksTestPredictedAll.npy', imgs_mask_test)
    
#     print('-' * 30)
#     print('Calculate mean dice coeff...')
    
#     mean = 0.0
#     for i in range(num_test):
#         mean += dice_coef_np(imgs_mask_test_true[i, 0], imgs_mask_test[i, 0])
#     mean /= num_test

    np.save('gdrive/My Drive/luna/masks_mask_test.npy', imgs_mask_test)
    
    print('-' * 30)
    print('Calculate mean dice coeff...')
    
    mean = 0.0
    for i in range(num_test):
        mean += dice_coef_np(imgs_mask_test_true[i, 0], imgs_mask_test[i, 0])
    mean /= num_test

#     print("Mean Dice Coeff : ", mean)

if __name__ == '__main__':
    train_and_predict(True)


------------------------------
Loading and preprocessing train data...
BATCH_SIZE : 8
EPOCHS : 5
------------------------------
Creating and compiling model...
Instructions for updating:
Colocations handled automatically by placer.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1, 512, 512)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 32, 512, 512)  320         input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 32, 512, 512)  9248        conv2d_1[0][0]                   
_____________________________________________________________